# Projeto Integrador 
Digital House Data Science

## Integrantes:

# Objetivo:
## Este projeto tem os seguintes objetivos:
### Realizar uma análise de carrinhos de uma plataforma de compra e comparação de preços. 
### Com base nesta análise faremos um sistema "predição" para o resultado do carrinho, ou seja, se será um carrinho que seguirá o fúnil de venda ou será abandonado.
### Por fim, será realizado um sistema de recomendação para transformar possíveis carrinhos abandonados em carrinhos de sucesso.

# Bases:
## Para este projeto serão utilizadas informações de carrinhos compostos durante o período de 2018 e 2019, estes carrinhos estão classificados em três tipos: Carrinho Abandonado, Aprovado, Cancelado/Recusado, Pendente. 
## O tipo de cliente a ser analisado será PJ. 
## Usaremos apenas a região de São Paulo, pois é a região com mais incidências.

In [1]:
#importando bibliotecas básicas

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


In [2]:
# Configurando o Pandas para a dimensão das tabelas

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)

In [6]:
# Importando os dados

df_compras = pd.read_csv(r'dados\dados_empresa\Tabela das Compras.csv')
df_consumidores = pd.read_csv(r'dados\dados_empresa\consumidores.csv')

In [7]:
# Dando uma olhada nas tabelas
df_compras.head()

,id_pedido,id_carrinho,status,desc_status,id_cliente,id_distribuidor,id_sku,id_produto,produto,anomes_carrinho,data_carrinho,anomes_status_pedido,data_status_pedido,quantidade,volumepedidos,volumecompras,volumependente,volumecanceladorecusado,meio_pagamento
0,107783,71935,0,Carrinho Abandonado,30696,137,4696.0,16544.0,PAPEL TOALHA INTERFOLHA 2 DOBRAS SOFT BRANCO P...,201812,2018-12-11 12:05:40.081189+00,NaN,NaN,4.0,59.96,NaN,NaN,NaN,DEPÓSITO EM CONTA ANTECIPADO
1,3,3,0,Carrinho Abandonado,3,15,NaN,NaN,- BALIX,201511,2015-11-11 20:28:39.699416+00,NaN,NaN,1.0,22.90,NaN,NaN,NaN,DESCONTINUADO
2,53780,33739,0,Carrinho Abandonado,3,4258,29813.0,1657959.0,CÁPSULAS DE CAFÉ ESPRESSO FORTISSIMO PARA MÁQU...,201804,2018-04-18 15:01:30.063939+00,NaN,NaN,1.0,14.50,NaN,NaN,NaN,DESCONTINUADO
3,53782,33737,0,Carrinho Abandonado,4583,2900,18244.0,637259.0,ESCOVA DENTAL MACIA ESSENCIAL CLEAN UNIDADE -...,201804,2018-04-18 14:48:41.568361+00,NaN,NaN,1.0,1.69,NaN,NaN,NaN,DESCONTINUADO
4,53773,33739,0,Carrinho Abandonado,3,1203,13021.0,61708.0,MACARRÃO ESPAGUETE (8) COM OVOS PACOTE 500G - ...,201804,2018-04-18 15:01:30.063939+00,NaN,NaN,24.0,45.12,NaN,NaN,NaN,DESCONTINUADO


In [12]:
df_compras.tail()

,id_pedido,id_carrinho,status,desc_status,user_id,id_distribuidor,id_sku,id_produto,produto,anomes_carrinho,data_carrinho,anomes_status_pedido,data_status_pedido,quantidade,volumepedidos,volumecompras,volumependente,volumecanceladorecusado,meio_pagamento
298697,110,122,4,Aprovado,3,33,NaN,NaN,- TECNUTRI,201511,2015-11-24 17:44:36.999996+00,201511.0,2015-11-24 17:44:36.999996+00,1.0,8.99,8.99,NaN,NaN,DESCONTINUADO
298698,72,82,4,Aprovado,3,33,NaN,NaN,- PICININ,201511,2015-11-17 15:17:10.499341+00,201511.0,2015-11-17 15:17:10.499341+00,6.0,92.70,92.70,NaN,NaN,DESCONTINUADO
298699,72,82,4,Aprovado,3,33,NaN,NaN,- NOBRE,201511,2015-11-17 15:17:10.499341+00,201511.0,2015-11-17 15:17:10.499341+00,5.0,61.25,61.25,NaN,NaN,DESCONTINUADO
298700,72,82,4,Aprovado,3,33,NaN,NaN,- VIGOR,201511,2015-11-17 15:17:10.499341+00,201511.0,2015-11-17 15:17:10.499341+00,12.0,105.96,105.96,NaN,NaN,DESCONTINUADO
298701,4,4,3,Cancelado/Recusado,3,15,NaN,NaN,- BALIX,201511,2015-11-11 20:33:45.601018+00,201511.0,2015-11-11 20:33:45.601018+00,10.0,339.00,NaN,NaN,339.0,DESCONTINUADO


In [8]:
df_consumidores.head()

,user_id,consumidor_id,cep,cidade,cidade_id,estado,estado_id,anomes_cadastro,data_cadastro,data_ultimo_login,data_aniversario_pf,cadastro_com_cidade,tipo
0,1642968,8494,NaN,NaN,NaN,NaN,NaN,201910,31-10-2019,31-10-2019,NaN,0,NaN
1,1642861,8493,11704-400,PRAIA GRANDE,5167.0,SÃO PAULO,26.0,201910,31-10-2019,31-10-2019,NaN,1,PJ
2,1642831,8492,NaN,NaN,NaN,NaN,NaN,201910,31-10-2019,31-10-2019,NaN,0,NaN
3,1642498,8491,13152-246,COSMÓPOLIS,4849.0,SÃO PAULO,26.0,201910,31-10-2019,31-10-2019,NaN,1,PJ
4,1640977,8490,05712-080,SÃO PAULO,5270.0,SÃO PAULO,26.0,201910,30-10-2019,30-10-2019,15-06-1982,1,PF


In [11]:
df_consumidores.tail()

,user_id,consumidor_id,cep,cidade,cidade_id,estado,estado_id,anomes_cadastro,data_cadastro,data_ultimo_login,data_aniversario_pf,cadastro_com_cidade,tipo
8083,10,5,NaN,NaN,NaN,NaN,NaN,201510,09-10-2015,09-10-2015,NaN,0,NaN
8084,9,4,NaN,NaN,NaN,NaN,NaN,201510,09-10-2015,09-10-2015,NaN,0,NaN
8085,5,3,13960-000,SOCORRO,5288.0,SÃO PAULO,26.0,201510,03-10-2015,26-06-2019,NaN,1,PJ
8086,3,2,13960-000,SÃO PAULO,5270.0,SÃO PAULO,26.0,201509,17-09-2015,30-10-2019,NaN,1,PJ
8087,1,1,56306-425,NaN,NaN,NaN,NaN,201509,16-09-2015,13-11-2015,NaN,0,NaN


In [9]:
# Vamos igualar a nomeação da coluna sobre o cliente pois já consultamos e tratam-se da mesma informação.
df_compras.rename(columns= {'id_cliente': 'user_id'}, inplace=True)

In [10]:
# Apenas verificando (computação não é uma área exata rs)
df_compras.columns

Index(['id_pedido', 'id_carrinho', 'status', 'desc_status', 'user_id', 'id_distribuidor', 'id_sku', 'id_produto', 'produto', 'anomes_carrinho', 'data_carrinho', 'anomes_status_pedido', 'data_status_pedido', 'quantidade', 'volumepedidos', 'volumecompras', 'volumependente', 'volumecanceladorecusado', 'meio_pagamento'], dtype='object')

In [14]:
df_consumidores.columns

Index(['user_id', 'consumidor_id', 'cep', 'cidade', 'cidade_id', 'estado', 'estado_id', 'anomes_cadastro', 'data_cadastro', 'data_ultimo_login', 'data_aniversario_pf', 'cadastro_com_cidade', 'tipo'], dtype='object')

In [21]:
# Vamos juntar as Tabelas para termos as informações relevantes (que no caso é apenas o tipo) em apenas um DataFrame
df_compras_consu = pd.merge(df_consumidores, df_compras, on = 'user_id')

In [23]:
df_compras_consu.head(2)

,user_id,consumidor_id,cep,cidade,cidade_id,estado,estado_id,anomes_cadastro,data_cadastro,data_ultimo_login,data_aniversario_pf,cadastro_com_cidade,tipo,id_pedido,id_carrinho,status,desc_status,id_distribuidor,id_sku,id_produto,produto,anomes_carrinho,data_carrinho,anomes_status_pedido,data_status_pedido,quantidade,volumepedidos,volumecompras,volumependente,volumecanceladorecusado,meio_pagamento
0,1642498,8491,13152-246,COSMÓPOLIS,4849.0,SÃO PAULO,26.0,201910,31-10-2019,31-10-2019,NaN,1,PJ,346752,1684898,0,Carrinho Abandonado,37014,44441.0,9128554.0,BOLINHO DE QUEIJO COM MASSA DE BATATA CONGELAD...,201910,2019-10-31 00:49:17.966112+00,NaN,NaN,10.0,79.6000,NaN,NaN,NaN,CARTÃO DE CRÉDITO À VISTA (ROTATIVO)
1,1642498,8491,13152-246,COSMÓPOLIS,4849.0,SÃO PAULO,26.0,201910,31-10-2019,31-10-2019,NaN,1,PJ,346751,1684898,0,Carrinho Abandonado,703,23895.0,5510121.0,MANDIOCA TOLETE CONGELADA PACOTE POR KG - PRON...,201910,2019-10-31 00:49:17.966112+00,NaN,NaN,12.0,102.9996,NaN,NaN,NaN,PAGAMENTO NA ENTREGA DA MERCADORIA


In [29]:
# Dropando os valores nulos de tipo
df_compras_consu["tipo"].dropna(inplace=True)
df_compras_consu["tipo"].isnull().value_counts()

False    267551
Name: tipo, dtype: int64

In [32]:
# Deixando apenas os carrinhos de SP
df_compras_consu = df_compras_consu[df_compras_consu['estado_id'] == 26.0]
df_compras_consu["estado_id"].value_counts()

26.0    266344
Name: estado_id, dtype: int64

In [33]:
df_compras_consu.head(2)

,user_id,consumidor_id,cep,cidade,cidade_id,estado,estado_id,anomes_cadastro,data_cadastro,data_ultimo_login,data_aniversario_pf,cadastro_com_cidade,tipo,id_pedido,id_carrinho,status,desc_status,id_distribuidor,id_sku,id_produto,produto,anomes_carrinho,data_carrinho,anomes_status_pedido,data_status_pedido,quantidade,volumepedidos,volumecompras,volumependente,volumecanceladorecusado,meio_pagamento
0,1642498,8491,13152-246,COSMÓPOLIS,4849.0,SÃO PAULO,26.0,201910,31-10-2019,31-10-2019,NaN,1,PJ,346752,1684898,0,Carrinho Abandonado,37014,44441.0,9128554.0,BOLINHO DE QUEIJO COM MASSA DE BATATA CONGELAD...,201910,2019-10-31 00:49:17.966112+00,NaN,NaN,10.0,79.6000,NaN,NaN,NaN,CARTÃO DE CRÉDITO À VISTA (ROTATIVO)
1,1642498,8491,13152-246,COSMÓPOLIS,4849.0,SÃO PAULO,26.0,201910,31-10-2019,31-10-2019,NaN,1,PJ,346751,1684898,0,Carrinho Abandonado,703,23895.0,5510121.0,MANDIOCA TOLETE CONGELADA PACOTE POR KG - PRON...,201910,2019-10-31 00:49:17.966112+00,NaN,NaN,12.0,102.9996,NaN,NaN,NaN,PAGAMENTO NA ENTREGA DA MERCADORIA


Por enquanto é isso que temos, vamos trabalhar um pouco nessa tabela, primeiro vamos alterar os status por um inteiro de 0 a 3 que transfere cada caso, portando, vamos seguir a seguinte regra:
* "Carrinho Abandonado": 0
* "Aprovado":1
* "Cancelado/Recusado": 2
* "Pendente": 3
* Caso tenha um caso fora desses: 99

In [42]:
status_final = []
for linha in df_compras_consu['desc_status']:
    if linha == "Carrinho Abandonado":
        status_final.append(0)
    elif linha == "Aprovado":
        status_final.append(1)
    elif linha == "Cancelado/Recusado":
        status_final.append(2)
    elif linha == "Pendente":
        status_final.append(3)
    else:
        status_final.append(99)
df_compras_consu['status_final_target'] = status_final
df_compras_consu['status_final_target'].value_counts()

0    214153
1     31792
2     19628
3       771
Name: status_final_target, dtype: int64

Com isso conseguimos ver a diferença de amostras que temos de cada status, no nosso caso, o target para nosso recomendador serão os carrinhos abandonados e pendentes.
* Será que ambos os status tem características semelhantes?
* Podemos considerar o Pendente como Abandonado?

In [41]:
df_compras_consu.columns

Index(['user_id', 'consumidor_id', 'cep', 'cidade', 'cidade_id', 'estado', 'estado_id', 'anomes_cadastro', 'data_cadastro', 'data_ultimo_login', 'data_aniversario_pf', 'cadastro_com_cidade', 'tipo', 'id_pedido', 'id_carrinho', 'status', 'desc_status', 'id_distribuidor', 'id_sku', 'id_produto', 'produto', 'anomes_carrinho', 'data_carrinho', 'anomes_status_pedido', 'data_status_pedido', 'quantidade', 'volumepedidos', 'volumecompras', 'volumependente', 'volumecanceladorecusado', 'meio_pagamento', 'status_final_target'], dtype='object')

Nosso DataFrame ainda tem muita informação desnecessária, vamos ter que limpas um pouco mais e avaliar se teremos valores faltando.

In [50]:
### Decidi manter o id_sku e volumepedidos pois não sei o que significam ainda.
df_limpo = df_compras_consu[['user_id', 'cidade_id', 'tipo', 'id_carrinho', 
                            'id_sku','produto', 'id_produto', 'quantidade', 'volumepedidos']]

In [51]:
# Ainda temos muitos valores nulos, vamos ter que cruzar isso com a tabela de produtos para poder resolver...
df_limpo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 266344 entries, 0 to 268620
Data columns (total 9 columns):
user_id          266344 non-null int64
cidade_id        266344 non-null float64
tipo             266344 non-null object
id_carrinho      266344 non-null int64
id_sku           200433 non-null float64
produto          266344 non-null object
id_produto       239062 non-null float64
quantidade       266185 non-null float64
volumepedidos    266066 non-null float64
dtypes: float64(5), int64(2), object(2)
memory usage: 20.3+ MB


In [52]:
# Vamos dar uma olhadinha em como ficou;
# Não gostei do index generico pq temos informações repetidas em algumas colunas...
df_limpo.head()

,user_id,cidade_id,tipo,id_carrinho,id_sku,produto,id_produto,quantidade,volumepedidos
0,1642498,4849.0,PJ,1684898,44441.0,BOLINHO DE QUEIJO COM MASSA DE BATATA CONGELAD...,9128554.0,10.0,79.6000
1,1642498,4849.0,PJ,1684898,23895.0,MANDIOCA TOLETE CONGELADA PACOTE POR KG - PRON...,5510121.0,12.0,102.9996
2,1642498,4849.0,PJ,1684898,4440.0,BATATA PALITO TRADICIONAL 10X10MM CONGELADA PO...,539148.0,15.0,105.7200
3,1642498,4849.0,PJ,1684898,44439.0,BOLINHO DE CARNE COM MASSA DE BATATA CONGELADO...,9128553.0,10.0,83.7000
4,1642498,4849.0,PJ,1684898,44457.0,KIBE DE BATATA COM CATUPIRY CONGELADO 25G 400...,9128565.0,10.0,84.2000


In [53]:
# Vamos transformar o id_carrinho em index, ele ainda não será único, mas assim que fizermos a matriz de produtos, será
df_limpo = df_limpo.set_index('id_carrinho')

In [54]:
df_limpo.head()

,user_id,cidade_id,tipo,id_sku,produto,id_produto,quantidade,volumepedidos
id_carrinho,,,,,,,,
1684898,1642498,4849.0,PJ,44441.0,BOLINHO DE QUEIJO COM MASSA DE BATATA CONGELAD...,9128554.0,10.0,79.6000
1684898,1642498,4849.0,PJ,23895.0,MANDIOCA TOLETE CONGELADA PACOTE POR KG - PRON...,5510121.0,12.0,102.9996
1684898,1642498,4849.0,PJ,4440.0,BATATA PALITO TRADICIONAL 10X10MM CONGELADA PO...,539148.0,15.0,105.7200
1684898,1642498,4849.0,PJ,44439.0,BOLINHO DE CARNE COM MASSA DE BATATA CONGELADO...,9128553.0,10.0,83.7000
1684898,1642498,4849.0,PJ,44457.0,KIBE DE BATATA COM CATUPIRY CONGELADO 25G 400...,9128565.0,10.0,84.2000


Próximos passos:
* Pegar os IDs dos produtos que estão faltando cruzando com a tabela de produtos.
* Pegar os IDs de SKU faltando com base na tabela de Sku x Produtos